In [1]:
from __future__ import print_function
%matplotlib inline
%pylab inline
# %load_ext memory_profiler
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy
import itertools
from tabulate import tabulate
from pomegranate import *
from pomegranate import BayesianNetwork
import iac_org as iacO

Populating the interactive namespace from numpy and matplotlib


In [2]:
attributes = ["gang", "age", "edu", "mar", "occ", "name"]

def prob(evidences, atts):
    count, total = 0, 0
    for datapoint in iacO.data:
        inc = True
        for key in evidences:
            inc = inc and datapoint[key] == evidences[key]
        if inc:
            total += 1
            f = True
            for key in atts:
                f = f and datapoint[key] == atts[key]
            if f:
                count += 1
    return count, total, float(count)/total

def enums(key):
    vals = set()
    for datapoint in iacO.data:
        if datapoint[key] not in vals:
            vals.add(datapoint[key])
    return list(vals)

In [3]:
import collections
noisy_p_feature_given_hidden = {}
c = 0
for name in enums(attributes[-1]):
    noisy_p_feature_given_hidden[name] = {}
    for att in attributes[:-1]:
        pl = 0.1/(len(enums(att))-1)
        for val in enums(att):
            noisy_p_feature_given_hidden[name][val] = 0.9 if prob({"name":name},{att:val})[2] == 1.0 else pl
            c += 1 if prob({"name":name},{att:val})[2] == 1 else 0
            
print(prob({"name":"George"},{"edu":"JH"})[2], prob({"name":"George"},{"edu":"COL"})[2], prob({"name":"George"},{"edu":"HS"})[2])
print(c/27, noisy_p_feature_given_hidden["George"]["JH"], noisy_p_feature_given_hidden["George"]["COL"], noisy_p_feature_given_hidden["George"]["HS"])

1.0 0.0 0.0
5 0.9 0.05 0.05


In [6]:
distributions = {}
for att in attributes:
    probs = {}
    vals = enums(att)
    for val in vals:
        probs[val] = prob({},{att:val})[2]
    distributions[att] = DiscreteDistribution(probs)

dataInArray = []
n = {}
for datapoint in iacO.data:
    row = []
    for att in attributes:
        row.append(datapoint[att])
    dataInArray.append(row)
    
src = []
dist = []
states = []
for att in attributes:
    src.append(enums(att))
    dist.append(distributions[att])
print(len(dataInArray))
# print(tabulate(dataInArray))

27


In [7]:
# p(h_i|e) = p(h_i)*mul_j(p(e_j|h_i))/sum_k(p(h_k)mul_j(p(e_j|h_k)))

Noisy_CPT = []
d_evidence = {}
for element in itertools.product(*src):
    name = element[-1]
    evidences = element[:-1]
    if evidences not in d_evidence:
        d_evidence[evidences] = {}
    if name not in d_evidence[evidences]:
        d_evidence[evidences][name] = 1
    for evi in evidences:
        d_evidence[evidences][name] *= noisy_p_feature_given_hidden[name][evi]
    
eq_evidence = {}
for element in itertools.product(*src[:-1]):
    evidences = element
    eq_evidence[evidences] = 0
    for name in enums("name"):
        eq_evidence[evidences] += d_evidence[evidences][name]

c = 0
Noisy_CPT_display = []
for element in itertools.product(*src):
    name = element[-1]
    evidences = element[:-1]
    temp = list(element)
    normalized_noisy_prob = d_evidence[evidences][name]/eq_evidence[evidences]
    Noisy_CPT.append(temp+[normalized_noisy_prob])
    if temp in dataInArray:
        Noisy_CPT_display.append(temp+[normalized_noisy_prob]+[1])
        c += 1
    else:
        Noisy_CPT_display.append(temp+[normalized_noisy_prob]+[0])

print(tabulate(Noisy_CPT_display, headers=attributes+["noisy prob", "actual"]))

gang    age    edu    mar       occ      name      noisy prob    actual
------  -----  -----  --------  -------  ------  ------------  --------
Jets    20s    HS     single    bookie   Pete     0.837869            1
Jets    20s    HS     single    bookie   Ned      1.59631e-05         0
Jets    20s    HS     single    bookie   Jim      0.000143667         0
Jets    20s    HS     single    bookie   Ralph    0.000143667         0
Jets    20s    HS     single    bookie   Nick     0.000287335         0
Jets    20s    HS     single    bookie   Clyde    0.00258601          0
Jets    20s    HS     single    bookie   Rick     1.59631e-05         0
Jets    20s    HS     single    bookie   John     0.000143667         0
Jets    20s    HS     single    bookie   Lance    0.000143667         0
Jets    20s    HS     single    bookie   Ken      0.00517203          0
Jets    20s    HS     single    bookie   Mike     0.00258601          0
Jets    20s    HS     single    bookie   Fred     0.0465483     

In [8]:
cond = ConditionalProbabilityTable(Noisy_CPT, dist[:-1])
name = State(cond, name="name")
gang = State(distributions["gang"], name="gang")
age = State(distributions["age"], name="age")
edu = State(distributions["edu"], name="edu")
mar = State(distributions["mar"], name="mar")
occ = State(distributions["occ"], name="occ")

model = BayesianNetwork("Jets and Sharks")
model.add_states(gang, age, edu, mar, occ, name)

model.add_edge(gang, name)
model.add_edge(age, name)
model.add_edge(edu, name)
model.add_edge(mar, name)
model.add_edge(occ, name)
model.bake()

In [10]:
# print(prob({'gang': 'Jets', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'occ': 'bookie' ,'mar': 'single'}))
# print(prob({'name': 'Lance', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'name': 'Lance', 'occ': 'pusher'}))
# print(model.predict_proba({'gang': 'Sharks', 'age': '20s'}))
# print(model.predict_proba({'gang': 'Jets'}))
print(model.predict_proba({'name': 'John'}))
# print(prob({},{'gang': 'Jets'}))
# print(prob({},{'name': 'John'x}))

[{
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "Jets" :0.7240157245139609,
            "Sharks" :0.27598427548603904
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "20s" :0.8487707173912195,
            "30s" :0.07859448957076018,
            "40s" :0.07263479303802024
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "HS" :0.1354702285052171,
            "JH" :0.7514641824409919,
            "COL" :0.11306558905379112
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "single" :0.13911558675091257,
            "married" :0.8337336434254842,
            "divorced" :0